# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
uk_food = mongo.uk_food

In [ ]:
# review the collections in our database
uk_food.list_collection_names()

In [ ]:
# assign the collection to a variable
establishments = uk_food.establishments

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
# Use count_documents to display the number of documents in the result
hygiene_amount = establishments.count_documents(query) 
print(f"Number of documents in results: {hygiene_amount}")
# Display the first document in the results using pprint
hygiene_results = list(establishments.find(query))
pprint(hygiene_results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(hygiene_results)
# Display the number of rows in the DataFrame
print(f"Rows in DataFrame: {len(hygiene_df)}")
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query ={
    'LocalAuthorityName': {'$regex': 'London'},
    'RatingValue': {'$gte': '4'}
}
ratings = list(establishments.find(query))
# Use count_documents to display the number of documents in the result
ratings_count = establishments.count_documents(query)
print(f"Number of documents in results: {ratings_count}")
# Display the first document in the results using pprint
pprint(ratings[0])

In [ ]:
# Convert the result to a Pandas DataFrame
ratings_df = pd.DataFrame(ratings)
# Display the number of rows in the DataFrame
print(f"Rows in DataFrame: {len(ratings_df)}")
# Display the first 10 rows of the DataFrame
ratings_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'}
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    'RatingValue': '5',
    'geocode.latitude':{
        '$gte': latitude - degree_search,
        '$lte': latitude + degree_search
    },
    'geocode.longitude':{
        '$gte': longitude - degree_search,
        '$lte': longitude + degree_search
    }
}
sort =  [('scores.Hygiene', 1)]
limit = 5

# Print the results
search_results = list(establishments.find(query).sort(sort).limit(limit))
pprint(search_results)

In [ ]:
# Convert result to Pandas DataFrame
search_df = pd.DataFrame(search_results)
search_df

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
match_query = {'$match': {'scores.Hygiene': 0}}

group_query = {
    '$group':{'_id': '$LocalAuthorityName','count': {'$sum': 1}}
}

fields = {
    'LocalAuthority': 1,
    'scores.Hygience': 1
}
# 3. Sorts the matches from highest to lowest
sort_query = {
    '$sort': {'count': -1}
}

pipeline = [match_query, group_query, sort_query]
pipeline_results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print(f"Number of documents in the result: {len(pipeline_results)}")
# Print the first 10 results
pprint(pipeline_results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
pipeline_df = pd.DataFrame(pipeline_results)
# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(pipeline_df)}")
# Display the first 10 rows of the DataFrame
pipeline_df.head(10)